<a href="https://colab.research.google.com/github/OmarReales/AssistantGPT/blob/main/AssistantAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U google-generativeai #dependencias geminiAI

In [21]:
import google.generativeai as genai
import textwrap
import sys
from google.colab import userdata
from abc import ABC, abstractmethod
from IPython.display import display
from IPython.display import Markdown

# Esta función se usa para dejar el formato Markdown que devuelve Gemini en formato compatible con Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
# Configuramos nuestra instancia del modelo con nuestra API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key = GOOGLE_API_KEY)

In [23]:
# Create the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  system_instruction="Eres un asistente de estudio inteligente creado para ayudar a los estudiantes a comprender y aprender una amplia variedad de temas. Tu objetivo es proporcionar explicaciones claras y concisas, responder preguntas, ayudar con la resolución de problemas y ofrecer consejos útiles para el estudio. Debes responder de manera amigable, paciente y comprensiva, adaptando tus respuestas al nivel de conocimiento del usuario. Evita el uso de jerga técnica innecesaria y proporciona ejemplos prácticos cuando sea posible. Si no tienes suficiente información para responder una pregunta, ofrécele al usuario sugerencias sobre cómo investigar más."
)

In [24]:
#cierra el programa
def cerrar():
    print ("Espero haber sido de ayuda! saludos")
    sys.exit()

# Función que solicita una respuesta S/N del usuario
def continua(texto_pregunta):
    resp = input(f'{texto_pregunta} [S/N]: ').upper()
    if resp == 'S':
        return True
    return False


# Función que devuelve un título centrado con guiones
def titulo(texto:str,largo:int=80):
    return f"{'-'*largo}\n{texto.title().center(largo)}\n{'-'*largo}"


# Función que calcula el largo de la opción más larga en un menú
def obtener_largo_opcion_mas_larga(tupla_opciones):
    maximo_largo = -float('inf')
    for i,texto in enumerate(tupla_opciones):
        if len(texto) > maximo_largo:
            maximo_largo = len(texto)
    return maximo_largo

# Función que verifica si una cadena es un entero
def isint(str_numero:str)->bool:
    try:                    # Intenta convertir str_numero a int
        int(str_numero)
    except:                 # Si no puede convertirlo devuelve False
        return False
    return True             # Si puede convertirlo devuelve True


# Función que lee un entero en un rango determinado
def leer_entero(mensaje:str='Ingrese un entero: ',minimo:int=-float('inf'),maximo:int=float('inf'))->int:
    todo_ok = False
    while not todo_ok:
        cadena = input(mensaje)
        if isint(cadena):
            numero = int(cadena)
            if minimo <= numero <= maximo:
                todo_ok = True
            else:
                print(f"Número {numero} fuera de rango [{minimo}] .. [{maximo}]")
        else:
            print(f"{cadena} No es un int.")
    return int(cadena)

# Función que muestra un menú con las opciones de una tupla
def opcion(tupla_opciones:str)->int:
    #La primera opcion es el titulo
    #Las demas son las opciones
    largo = obtener_largo_opcion_mas_larga(tupla_opciones)
    for index,opcion in enumerate(tupla_opciones):
        if index == 0:
            print(titulo(opcion,largo))
        else:
            print(opcion.title())
    return leer_entero("Ingrese una opcion: ",1,4)

In [25]:
class studyAssistant(ABC):
    def __init__(self) -> None:
        self.__nombre = "AssistantGPT"
        self.__respuesta = None
        self.__consulta = None
        self.__OPCIONES = ["Escoge alguna opcion",
        "1. Asistente de estudio",
        "2. Salir del programa"
        ]
        self._OPCIONES_ASISTENTE =["Escoje alguna opcion",
        "1. Quiero que me expliques sobre un tema",
        "2. Quiero un resumen de un texto",
        "3. Salir del programa"
        ]

#Este metodo es para preguntarle al usuario si tiene una historia o crear una historia en el momento y trabajar sobre ello
    def consultar_asistente(self) -> None:
        opciones = self._OPCIONES_ASISTENTE
        opcion_seleccionada = opcion(opciones)

        if opcion_seleccionada == 1:
            self.explicar_tema()
        elif opcion_seleccionada == 2:
            self.crear_resumen()
        elif opcion_seleccionada == 3:
            cerrar()
        else:
            raise ValueError("Marque una opción válida")

#Metodo para crear una historia segun las especificaciones del usuario
    def crear_resumen (self) ->str:
        inputText = input("Indiqua el tema que quieras que resuma: ")

        contexto = f"Creame un resumen de no mas de 500 palabras sobre: '{inputText}' "
        conversacion = [{"role": "user", "content": contexto}]

        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(f"{inputText} resume la respuesta en 500 palabras maximo y que sea facil de identificar los puntos claves")

        generated_text = response.candidates[0].content.parts[0].text
        print("AssistantGPT: \n", generated_text)

        self.__resumen = generated_text

        if continua("Quieres hacer una consulta?"):
            self._interfaz()
        else:
            cerrar()


# Método para obtener y enviar el resumen al asistente de estudio
    def mandar_resumen(self) ->str:
        self.__resumen = input("Escribe el texto a resumir: ")
        print(f"\n El texto que has ingresado es: \n''{self.__resumen}''")
        return self.__resumen

# Método para procesar el resumen y realizar consultas
    def procesar_resumen(self) -> None:
        self.__consulta = self.consulta() #Ejemplo "Quiero que me hagas una lluvia de ideas para continuar la historia. Como maximo 2 respuestas"

        contexto = f"{self.__resumen}\nUsuario: {self.__consulta}"
        conversacion = [{"role": "user", "content": contexto}]

        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(f"{self.__consulta}")

        generated_text = response.candidates[0].content.parts[0].text
        print("AssistantGPT: \n", generated_text)

        if continua("Alguna otra consulta?"):
            self._interfaz()
        else:
            cerrar()
# Método para explicar un tema y realizar consultas
    def explicar_tema(self) -> None:
        tema = input("Indica el tema que quieres que te explique: ")
        contexto = f"Explicame el siguiente tema: '{tema}'"
        conversacion = [{"role": "user", "content": contexto}]

        try:
            model = genai.GenerativeModel('gemini-1.5-flash')
            response = model.generate_content(contexto)
            generated_text = response.candidates[0].content.parts[0].text
            print("AssistantGPT: \n", generated_text)
        except Exception as e:
            print(f"An error occurred: {e}")

        if continua("Quiere hacer una consulta?"):
            self._interfaz()
        else:
            cerrar()

# Método para recibir la consulta del usuario
    def consulta(self) -> str:
        return str(input("\n Escriba la solicitud que desee realizar: "))

    def __str__ (self) ->str:
        return titulo("Bienvenido al Asistente de estudio")

    def nombre (self) ->str:
        return self.__nombre

    def _interfaz(self) ->None:
        self.__respuesta = opcion(self.__OPCIONES)

        if self.__respuesta == 1:
            self.consultar_asistente()
        elif self.__respuesta == 2:
            cerrar()
        else:
            raise ValueError("Marque una opción válida")

In [26]:
# Defino la parte principal del programa para que pueda iniciarse
def main():
    asistente = studyAssistant()

    while True:
        print(titulo(f"Bienvenido a *{asistente.nombre()}*"))
        asistente.consultar_asistente()
        asistente._interfaz()

# Verificar si este archivo es el punto de entrada principal
if __name__ == "__main__":
    main()

--------------------------------------------------------------------------------
                          Bienvenido A *Assistantgpt*                           
--------------------------------------------------------------------------------
----------------------------------------
          Escoje Alguna Opcion          
----------------------------------------
1. Quiero Que Me Expliques Sobre Un Tema
2. Quiero Un Resumen De Un Texto
3. Salir Del Programa
Ingrese una opcion: 1
Indica el tema que quieres que te explique: calentamiento global
AssistantGPT: 
 ## El calentamiento global: Una crisis planetaria

El calentamiento global se refiere al **aumento a largo plazo de la temperatura promedio de la Tierra**, provocado principalmente por la **emisión de gases de efecto invernadero** a la atmósfera. Estos gases atrapan el calor del sol, provocando un aumento de la temperatura global.

**Aquí algunos puntos clave sobre el calentamiento global:**

**1. Causas:**
* **Emisiones de gases d

SystemExit: 